In [ ]:
# Essential imports for CNN image applications
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from skimage import data, feature, filters, segmentation, color
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from PIL import Image
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Ready to explore CNN applications on real images!")
print(f"TensorFlow version: {tf.__version__}")
print(f"OpenCV version: {cv2.__version__}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# Load and explore different image types
def load_sample_images():
    """Load various types of sample images for demonstration"""
    
    # Load built-in sample images from scikit-image
    images = {
        'Cameraman': data.camera(),           # Grayscale
        'Astronaut': data.astronaut(),        # RGB
        'Coins': data.coins(),                # Grayscale
        'Coffee': data.coffee(),              # RGB
        'Chelsea (Cat)': data.chelsea(),      # RGB
    }
    
    return images

# Load sample images
sample_images = load_sample_images()

# Analyze image properties
def analyze_image_properties(image, name):
    """Analyze and display properties of an image"""
    print(f"\n📊 {name} Image Analysis:")
    print("-" * 40)
    print(f"Shape: {image.shape}")
    print(f"Data type: {image.dtype}")
    print(f"Min value: {image.min()}")
    print(f"Max value: {image.max()}")
    print(f"Mean value: {image.mean():.2f}")
    print(f"Memory usage: {image.nbytes} bytes")
    
    if len(image.shape) == 2:
        print("Type: Grayscale")
        print(f"Unique values: {len(np.unique(image))}")
    else:
        print(f"Type: {image.shape[2]}-channel color")
        for i in range(image.shape[2]):
            channel_name = ['Red', 'Green', 'Blue'][i] if image.shape[2] == 3 else f'Channel {i}'
            print(f"{channel_name} - Min: {image[:,:,i].min()}, Max: {image[:,:,i].max()}, Mean: {image[:,:,i].mean():.2f}")

# Display sample images with their properties
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Digital Image Types and Properties', fontsize=16, fontweight='bold')

for idx, (name, img) in enumerate(sample_images.items()):
    row = idx // 3
    col = idx % 3
    
    if len(img.shape) == 2:  # Grayscale
        axes[row, col].imshow(img, cmap='gray')
    else:  # Color
        axes[row, col].imshow(img)
    
    axes[row, col].set_title(f'{name}\nShape: {img.shape}')
    axes[row, col].axis('off')
    
    # Analyze properties
    analyze_image_properties(img, name)

# Hide the last subplot if we have an odd number of images
if len(sample_images) % 3 != 0:
    axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

# Demonstrate color space conversions
def demonstrate_color_spaces(rgb_image, image_name):
    """Demonstrate different color space representations"""
    
    # Convert to different color spaces
    hsv_image = color.rgb2hsv(rgb_image)
    lab_image = color.rgb2lab(rgb_image)
    gray_image = color.rgb2gray(rgb_image)
    
    # Create visualization
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'Color Space Conversions: {image_name}', fontsize=14, fontweight='bold')
    
    # Original RGB
    axes[0, 0].imshow(rgb_image)
    axes[0, 0].set_title('Original RGB')
    axes[0, 0].axis('off')
    
    # RGB channels separately
    for i, channel in enumerate(['Red', 'Green', 'Blue']):
        axes[0, i+1].imshow(rgb_image[:,:,i], cmap='gray')
        axes[0, i+1].set_title(f'{channel} Channel')
        axes[0, i+1].axis('off')
    
    # Grayscale
    axes[1, 0].imshow(gray_image, cmap='gray')
    axes[1, 0].set_title('Grayscale')
    axes[1, 0].axis('off')
    
    # HSV channels
    hsv_channels = ['Hue', 'Saturation', 'Value']
    for i, channel in enumerate(hsv_channels):
        axes[1, i+1].imshow(hsv_image[:,:,i], cmap='gray')
        axes[1, i+1].set_title(f'HSV {channel}')
        axes[1, i+1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return hsv_image, lab_image, gray_image

# Demonstrate color space conversions on astronaut image
if 'Astronaut' in sample_images:
    astronaut_img = sample_images['Astronaut']
    hsv_img, lab_img, gray_img = demonstrate_color_spaces(astronaut_img, 'Astronaut')

print("\n🔍 Key Insights about Digital Images:")
print("• Grayscale images are memory efficient and sufficient for many tasks")
print("• RGB images provide full color information but require 3x memory")
print("• Different color spaces highlight different image properties")
print("• Image preprocessing often involves converting between color spaces")
print("• CNNs can work with any color space, but RGB is most common")


In [ ]:
# Load and prepare CIFAR-10 dataset for comparison
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to categorical
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

print("📊 CIFAR-10 Dataset Overview:")
print(f"Training images: {X_train.shape}")
print(f"Test images: {X_test.shape}")
print(f"Number of classes: {len(class_names)}")
print(f"Classes: {class_names}")

# Visualize sample images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('CIFAR-10 Sample Images', fontsize=14, fontweight='bold')

for i in range(10):
    row = i // 5
    col = i % 5
    
    # Find first occurrence of each class
    class_idx = np.where(y_train == i)[0][0]
    
    axes[row, col].imshow(X_train[class_idx])
    axes[row, col].set_title(f'{class_names[i]}')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

# Traditional Computer Vision Approach using Hand-crafted Features
def extract_traditional_features(images):
    """Extract traditional computer vision features"""
    features_list = []
    
    for img in images:
        # Convert to grayscale for traditional methods
        if len(img.shape) == 3:
            gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
        else:
            gray = (img * 255).astype(np.uint8)
        
        feature_vector = []
        
        # 1. Color histogram features (using original color image)
        if len(img.shape) == 3:
            for channel in range(3):
                hist = cv2.calcHist([img[:,:,channel]], [0], None, [16], [0, 1])
                feature_vector.extend(hist.flatten())
        
        # 2. Texture features (Local Binary Pattern)
        try:
            lbp = feature.local_binary_pattern(gray, P=8, R=1, method='uniform')
            lbp_hist, _ = np.histogram(lbp, bins=16, range=(0, 16))
            feature_vector.extend(lbp_hist)
        except:
            # Fallback if LBP fails
            feature_vector.extend(np.zeros(16))
        
        # 3. Edge features
        edges = feature.canny(gray, sigma=1)
        edge_density = np.sum(edges) / edges.size
        feature_vector.append(edge_density)
        
        # 4. Basic statistical features
        feature_vector.extend([
            np.mean(gray),
            np.std(gray),
            np.min(gray),
            np.max(gray)
        ])
        
        features_list.append(feature_vector)
    
    return np.array(features_list)

# Extract features for a subset (to speed up computation)
n_samples = 5000
indices = np.random.choice(len(X_train), n_samples, replace=False)
X_train_subset = X_train[indices]
y_train_subset = y_train[indices]

print("\\n🔧 Extracting traditional computer vision features...")
traditional_features = extract_traditional_features(X_train_subset)
traditional_features_test = extract_traditional_features(X_test[:1000])  # Subset for testing

print(f"Traditional features shape: {traditional_features.shape}")

# Train traditional classifier (Random Forest)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

traditional_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
traditional_classifier.fit(traditional_features, y_train_subset.ravel())

# Predict with traditional method
trad_predictions = traditional_classifier.predict(traditional_features_test)
trad_accuracy = accuracy_score(y_test[:1000].ravel(), trad_predictions)

print(f"\\n📊 Traditional CV Accuracy: {trad_accuracy:.4f}")

# Create and train CNN
def create_cnn_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Train CNN (on subset for faster training)
cnn_model = create_cnn_model()
cnn_model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

print("\\n🧠 Training CNN...")
cnn_history = cnn_model.fit(
    X_train_subset, keras.utils.to_categorical(y_train_subset, 10),
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Evaluate CNN
cnn_test_loss, cnn_accuracy = cnn_model.evaluate(
    X_test[:1000], keras.utils.to_categorical(y_test[:1000], 10), 
    verbose=0
)

print(f"\\n📊 CNN Accuracy: {cnn_accuracy:.4f}")

# Compare results
comparison_data = {
    'Method': ['Traditional CV', 'CNN'],
    'Accuracy': [trad_accuracy, cnn_accuracy],
    'Features': ['Hand-crafted', 'Learned'],
    'Training Time': ['Fast', 'Slow'],
    'Generalization': ['Limited', 'Excellent']
}

print("\\n🏆 Method Comparison:")
print("=" * 50)
for method, acc in zip(comparison_data['Method'], comparison_data['Accuracy']):
    print(f"{method:15}: {acc:.4f} ({acc*100:.2f}%)")

print("\\n💡 Key Insights:")
print("• CNNs automatically learn optimal features for the task")
print("• Traditional methods require domain expertise for feature engineering")
print("• CNNs generalize better to new, unseen data")
print("• Traditional methods are faster to train but plateau in performance")
